In [16]:
%matplotlib inline
import seaborn as sns

In [17]:
from IPython.display import HTML

In [18]:
from __future__ import unicode_literals
from __future__ import division
from __future__ import print_function

import os

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 300)
pd.set_option("max_colwidth",1000)


from collections import defaultdict

import munch

from IPython.display import display, HTML

from spartan.utils.annotations.ensembl.gff3 import parse_gff3_attributes
from spartan.utils.files import tableFile2namedTuple

# Files

In [19]:
knearest_ALL_ = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/gmm_to_gff_pipeline/pipeline_runs/gmm_to_gff_testing_filterPSL_100bp/get_nearest_k_features/snp_list_afterFS_with_p_value_LD_1.snps_in_features.xls"
knearest_MS_ = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/gmm_to_gff_pipeline/pipeline_runs/gmm_to_gff_testing_filterPSL_100bp/get_nearest_k_features/snp_list_MS_1_with_p_value.snps_in_features.xls"
knearest_NB_ = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/gmm_to_gff_pipeline/pipeline_runs/gmm_to_gff_testing_filterPSL_100bp/get_nearest_k_features/snp_list_NB_1_with_p_value.snps_in_features.xls"
knearest_OT_ = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/gmm_to_gff_pipeline/pipeline_runs/gmm_to_gff_testing_filterPSL_100bp/get_nearest_k_features/snp_list_OT_1_with_p_value.snps_in_features.xls"

In [20]:
knearest_ALL = pd.read_excel(knearest_ALL_)
knearest_MS = pd.read_excel(knearest_MS_)
knearest_NB = pd.read_excel(knearest_NB_)
knearest_OT = pd.read_excel(knearest_OT_)

knearest = {'knearest_ALL': knearest_ALL,
            'knearest_MS': knearest_MS,
            'knearest_NB': knearest_NB,
            'knearest_OT': knearest_OT}


In [21]:
# l = pd.Series([10,30,7,83,14,5,120,86,136,92,48,8,188,7,7,7,143,8,18,65,51,24,64,5,95,99,124,68,99,11,39,5,5,59,223,7,33,69,176,31,126,102,71,38,31,28,84,103,15,102,5,50,28,71,178,528,243,723,122,661,294,270,353,2369,153,151,569,33,182,126,74,103])

In [22]:
# sns.rugplot(l);

In [23]:
for name, df in knearest.items():
    df["source"] = name

In [24]:
knearest_combo = pd.concat(knearest.values())

In [25]:
knearest_combo.head()

,SNP_chrom,SNP_start,SNP_end,feature_set_name,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts,distance,source
0,Scaffold106,349186,349187,official_annotations,Scaffold106,339112,350126,GFUI002898,0,+,339112,350123,"0,0,0",6,"6,123,129,50,252,211","0,1071,5842,6164,8068,10803",0,knearest_MS
1,Scaffold22,1291804,1291805,novel_mapped_tx,Scaffold22,1263294,1292230,TCONS_00034486,0,+,1263294,1301363,"0,0,0",23,"11,23,70,22,41,80,39,24,216,185,165,139,156,159,377,116,124,7,24,91,26,27,14,","1263294,1263308,1263352,1263426,1263453,1263506,1263602,1263642,1292555,1292848,1293098,1293407,1298373,1298603,1298828,1300837,1301022,1301159,1301167,1301191,1301292,1301322,1301349,",0,knearest_MS
2,Scaffold22,1291804,1291805,novel_mapped_tx,Scaffold22,1263294,1292230,TCONS_00034487,0,+,1263294,1301363,"0,0,0",27,"11,23,70,22,41,80,39,24,15,54,132,159,204,185,165,139,156,159,377,116,124,7,24,91,26,27,14,","1263294,1263308,1263352,1263426,1263453,1263506,1263602,1263642,1266675,1266943,1267008,1267168,1292567,1292848,1293098,1293407,1298373,1298603,1298828,1300837,1301022,1301159,1301167,1301191,1301292,1301322,1301349,",0,knearest_MS
3,Scaffold22,1291804,1291805,novel_mapped_tx,Scaffold22,1263294,1292230,TCONS_00034488,0,+,1263294,1301363,"0,0,0",23,"11,23,70,22,41,80,39,24,216,185,165,139,156,159,446,116,124,7,24,91,26,27,14,","1263294,1263308,1263352,1263426,1263453,1263506,1263602,1263642,1292555,1292848,1293098,1293407,1298373,1298603,1298828,1300837,1301022,1301159,1301167,1301191,1301292,1301322,1301349,",0,knearest_MS
4,Scaffold22,1291804,1291805,novel_mapped_tx,Scaffold22,1263294,1292230,TCONS_00034489,0,+,1263294,1301363,"0,0,0",32,"11,23,70,22,41,80,39,24,43,11,11,7,4,31,54,132,159,204,185,165,139,156,159,377,116,124,7,24,91,26,27,14,","1263294,1263308,1263352,1263426,1263453,1263506,1263602,1263642,1266824,1266871,1266883,1266894,1266901,1266912,1266943,1267008,1267168,1292567,1292848,1293098,1293407,1298373,1298603,1298828,1300837,1301022,1301159,1301167,1301191,1301292,1301322,1301349,",0,knearest_MS


In [26]:
results_ = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/gmm_to_gff_pipeline/pipeline_runs/gmm_to_gff_testing_filterPSL_100bp/snps_near_homologous_de/snps_near_homologous_de_distance_0.csv"
bed_ = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/gmm_to_gff_pipeline/pipeline_prep/produced/Glossina-fuscipes-IAEA_BASEFEATURES_GfusI1.3.sorted.bed"
fanno = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/functional/GfusI1.1_pre/argot2_out/argot_functional_annotations_ts150.h5"

# Link Templates

In [27]:
url = "https://www.vectorbase.org/Glossina_fuscipes/Location/View?r=[chrom]:[left]-[right]"

# Vectorbase Link Templates
gene_page          = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Gene/Summary?db=core;g=%(gene_name)s"> %(gene_name)s: gene home page</a>'''
protein_summary    = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/ProteinSummary?db=core;g=%(gene_name)s"> %(gene_name)s: protein summary </a>'''
gene_region_detail = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Location/View?db=core;g=%(gene_name)s;r"> %(gene_name)s: genome browser </a>'''
gene_ontology_bp   = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/Ontology/biological_process?db=core;g=%(gene_name)s;oid=biological_process"> %(gene_name)s: GO: biological process </a>'''
gene_ontology_cp   = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/Ontology/cellular_component?db=core;g=%(gene_name)s;oid=cellular_component"> %(gene_name)s: GO: cellular component </a>'''
gene_ontology_mf   = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/Ontology/molecular_function?db=core;g=%(gene_name)s;oid=molecular_function"> %(gene_name)s: GO: molecular function </a>'''

# Functions

In [28]:
def create_multiindex(df, levels, data_cols=None):
    
    # select data columns
    if data_cols is None:
        mi = df[[c for c in data.columns if c not in levels]].copy()
    else:
        assert isinstance(data_cols, list)
        mi = df[data_cols]
    
    # get multiindex values
    indexes = [df[i].copy() for i in levels]
    
    # add new indexes to data
    mi.index = indexes
    
    return mi




bed_headers = ["chrom",
               "chromStart",
               "chromEnd",
               "name",
               "score",
               "strand",
               "thickStart",
               "thickEnd",
               "itemRgb",
               "blockCount",
               "blockSizes",
               "blockStarts"]


def add_url_col(results, url, bed, bed_headers):
    """Add url column to results df."""
    print("begun adding urls.")
    # correct the templating of the url required by snakemake interpreting it as wildcards
    url = url.replace("[chrom]","{chrom}").replace("[left]","{left}").replace("[right]","{right}")

    bed = pd.read_csv(bed, sep='\t',names=bed_headers)[["chrom", "chromStart", "chromEnd", "name"]]
    bed = bed.rename(columns={"name":"gene_id_internal"})

    combo = pd.merge(left=results, right=bed, how='inner',
                     on="gene_id_internal", left_on=None, right_on=None)

    # count_cols = ['chromStart','chromEnd','SNP_start','SNP_end']
    combo['left'] = combo.apply(lambda row: min(row[['chromStart','chromEnd','SNP_start','SNP_end']]), axis=1)
    combo['right'] = combo.apply(lambda row: max(row[['chromStart','chromEnd','SNP_start','SNP_end']]), axis=1)
    combo['url'] = combo.apply(lambda row: url.format(chrom=row["chrom"],left=row["left"],right=row["right"]), axis=1)

    results = pd.merge(left=results, right=combo[["gene_id_internal","url"]], how='inner',
                     on="gene_id_internal", left_on=None, right_on=None)

    return results.drop_duplicates()

In [29]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for index, row in enumerate(self):
            html.append("<tr>")
            
            for col in row:
                # if header, format as such
                if index == 0:
                    html.append("<th>{0}</th>".format(col))
                else:
                    html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

def present_gene(gene_name, snp_coords, argot_df):
    fset = ','.join(list(snp_coords.query(""" gene_id_internal == '{gene}' """.format(gene=gene)).feature_set_name.values))
    display(HTML("<h2>{gene}({fset})</h2>".format(fset=fset,gene=gene_name)))
    
    nearby_snps = list(snp_coords.query(""" gene_id_internal == '{gene}' """.format(gene=gene)).snp_str.values)
    display(HTML("<b>SNPs:</b><br>%s<br>" % ("<br>".join(nearby_snps))))
    
    display(gene_table(gene_name=gene_name,
                       argot_df=argot_df,
                       tscore=200))
    
    display(HTML("<p>"))
    display(vb_link(gene_name=gene_name, link_template=gene_page))
    display(vb_link(gene_name=gene_name, link_template=protein_summary))
    display(vb_link(gene_name=gene_name, link_template=gene_region_detail))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology_bp))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology_cp))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology_mf))
    display(HTML("<hr>"))

In [30]:
def filter_by_gene(gene, argot_df):
    return argot_df[argot_df.Sequence.str.startswith(gene)]

def sort_by_TS(argot_df):
    return argot_df.sort_values(by='Total Score', ascending=0)

def filter_by_TS(tscore, argot_df):
    return argot_df[argot_df["Total Score"] >= tscore]

def gene_table(gene_name, argot_df, tscore):
    return sort_by_TS(filter_by_TS(tscore, filter_by_gene(gene_name, argot_df)))

def vb_link(gene_name, link_template):
    return HTML(link_template % dict(gene_name=gene_name))

def format_snps(snp_df):
    snp_line = "{chrom}:{end}"
    
    snp_df['snp_str'] = snp_df.apply(lambda row: snp_line.format(chrom=row.SNP_chrom, end=row.SNP_end), axis=1)

# Load inputs

In [54]:
data = knearest_combo.query(""" feature_set_name == 'official_annotations' """).copy()
# data.head()

In [56]:
data['SNP_id'] = data.apply(lambda row: '{chrom}:{end}'.format(chrom=row.SNP_chrom, end=row.SNP_end), axis=1)
data = data.rename(columns={"name": "gene_id_internal"})

In [57]:
data.head()

,SNP_chrom,SNP_start,SNP_end,feature_set_name,chrom,chromStart,chromEnd,gene_id_internal,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts,distance,source,SNP_id
0,Scaffold106,349186,349187,official_annotations,Scaffold106,339112,350126,GFUI002898,0,+,339112,350123,"0,0,0",6,"6,123,129,50,252,211","0,1071,5842,6164,8068,10803",0,knearest_MS,Scaffold106:349187
17,Scaffold27,959994,959995,official_annotations,Scaffold27,930934,962675,GFUI021833,0,+,930934,945628,"0,0,0",4,"1660,505,625,17896","0,8012,12986,13845",0,knearest_MS,Scaffold27:959995
18,Scaffold3,2235177,2235178,official_annotations,Scaffold3,2226587,2245585,GFUI030827,0,-,2226590,2245585,"0,0,0",6,"36,163,268,64,94,20","0,385,13446,14863,15521,18978",0,knearest_MS,Scaffold3:2235178
0,Scaffold137,615406,615407,official_annotations,Scaffold137,608555,619673,GFUI007441,0,-,616231,619281,"0,0,0",8,"381,467,63,2563,395,179,401,69","0,443,1949,7066,9702,10161,10409,11049",0,knearest_ALL,Scaffold137:615407
1,Scaffold144,593215,593216,official_annotations,Scaffold144,567694,601114,GFUI008496,0,-,567697,601114,"0,0,0",9,"102,38,65,50,73,129,114,7,61","0,744,3289,7045,13662,15174,27631,29146,33359",0,knearest_ALL,Scaffold144:593216


# Functional inference

In [58]:
argot2 = pd.read_hdf(path_or_buf=fanno, key='dataframe')

In [59]:
argot2.head()

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
0,GFUI034947-PA,P,GO:0006508,proteolysis,270.313447,0.496543,8.247696
1,GFUI035874-PA,F,GO:0005515,protein binding,529.038456,0.500000,5.471582
2,GFUI033625-PA,P,GO:0005980,glycogen catabolic process,307.758251,0.284597,13.424450
3,GFUI033625-PA,F,GO:0004134,4-alpha-glucanotransferase activity,159.513252,0.177063,10.153643
4,GFUI033625-PA,F,GO:0004135,"amylo-alpha-1,6-glucosidase activity",184.352303,0.177063,11.734746


In [66]:
snp_coords = data[["SNP_chrom","SNP_start","SNP_end","gene_id_internal","feature_set_name"]].drop_duplicates()

In [67]:
# snp_coords

In [68]:
format_snps(snp_coords)
# snp_coords

In [69]:
snp_coords

,SNP_chrom,SNP_start,SNP_end,gene_id_internal,feature_set_name,snp_str
0,Scaffold106,349186,349187,GFUI002898,official_annotations,Scaffold106:349187
17,Scaffold27,959994,959995,GFUI021833,official_annotations,Scaffold27:959995
18,Scaffold3,2235177,2235178,GFUI030827,official_annotations,Scaffold3:2235178
0,Scaffold137,615406,615407,GFUI007441,official_annotations,Scaffold137:615407
1,Scaffold144,593215,593216,GFUI008496,official_annotations,Scaffold144:593216
2,Scaffold175,347131,347132,GFUI011907,official_annotations,Scaffold175:347132
7,Scaffold33,1387925,1387926,GFUI026687,official_annotations,Scaffold33:1387926
8,Scaffold48,1013372,1013373,GFUI035780,official_annotations,Scaffold48:1013373
9,Scaffold8,1744225,1744226,GFUI051070,official_annotations,Scaffold8:1744226
0,Scaffold102,330091,330092,GFUI002233,official_annotations,Scaffold102:330092


In [70]:
for gene in sorted(list(snp_coords.gene_id_internal.values)):

    present_gene(gene_name=gene, snp_coords=snp_coords, argot_df=argot2)

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


...

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
19117,GFUI043027-PA,P,GO:0006260,DNA replication,5253.270634,0.492974,8.821469
19120,GFUI043027-PA,F,GO:0003887,DNA-directed DNA polymerase activity,4539.992874,0.322161,7.169783
19118,GFUI043027-PA,F,GO:0003677,DNA binding,2533.271379,0.250152,3.831010
19123,GFUI043027-PA,C,GO:0008622,epsilon DNA polymerase complex,2479.352690,0.332648,12.834245
19116,GFUI043027-PA,P,GO:0006261,DNA-dependent DNA replication,2445.618757,0.259546,10.374902
19122,GFUI043027-PA,C,GO:0005634,nucleus,1685.464129,0.804198,5.725783
19124,GFUI043027-PA,C,GO:0043231,intracellular membrane-bounded organelle,792.375979,0.946265,4.246234
19119,GFUI043027-PA,F,GO:0016740,transferase activity,569.214978,0.680699,2.487400
19121,GFUI043027-PA,F,GO:0016779,nucleotidyltransferase activity,409.609294,0.490394,5.055132


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
51722,GFUI043720-PA,P,GO:0006470,protein dephosphorylation,2077.021281,0.236174,11.013535
51725,GFUI043720-PA,P,GO:0035335,peptidyl-tyrosine dephosphorylation,1854.192243,0.104459,11.690944
51724,GFUI043720-PA,P,GO:0016311,dephosphorylation,1327.332234,0.334633,9.783756
51729,GFUI043720-PA,F,GO:0004725,protein tyrosine phosphatase activity,1106.817502,0.176805,8.510896
51726,GFUI043720-PA,F,GO:0016787,hydrolase activity,1088.596949,0.857645,2.643033
51730,GFUI043720-PA,F,GO:0008138,protein tyrosine/serine/threonine phosphatase activity,904.445986,0.147903,10.023433
51728,GFUI043720-PA,F,GO:0004721,phosphoprotein phosphatase activity,837.540183,0.541206,7.887060
51727,GFUI043720-PA,F,GO:0016791,phosphatase activity,663.461735,0.703193,6.651616
51733,GFUI043720-PA,C,GO:0005634,nucleus,555.072824,0.429894,5.725783
51732,GFUI043720-PA,C,GO:0005737,cytoplasm,497.846080,0.152037,3.178476


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
